In [1]:
# !pip install selenium
# !pip install bs4
# chromedriver download link : https://chromedriver.chromium.org/downloads

In [2]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from math import *

In [3]:
# 확인하고 싶은 아이디 입력
summoner_name = 'bbo bbo'

In [4]:
# 확인하고 싶은 자료 선정
Damage = []
CS = []
Tower = []
Wardplaced = []
visionWardplaced = []
WardKilled = []
Kill = []
Death = []
Assist = []
Level = []
Gold = []
#
Gamelength = []

In [5]:
# selenium을 백그라운드로 실행하기 위해 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [6]:
# selenium으로 실행시킬 url설정
summoner_url = 'https://poro.gg/summoner/kr/' + summoner_name
chrome_driver_path = "C:/Users/HEESU/[opgg데이터분석가]/chromedriver"
driver = webdriver.Chrome(chrome_driver_path)

driver.get(summoner_url)

<ipython-input-6-4db4e85b1dc4>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path)


In [7]:
# 솔로랭크의 자료만을 원하므로, 솔로랭크 버튼 클릭
solo_rank_button = driver.find_element_by_css_selector("#vue-profile > div.match-history-container.mt-3 > div:nth-child(1) > div.summoner-box__content > div > div.match-history-filter__queue-types > button:nth-child(2)")
solo_rank_button.click()
time.sleep(2)

<ipython-input-7-0c252052034b>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  solo_rank_button = driver.find_element_by_css_selector("#vue-profile > div.match-history-container.mt-3 > div:nth-child(1) > div.summoner-box__content > div > div.match-history-filter__queue-types > button:nth-child(2)")


In [8]:
# 더보기 버튼을 눌러야 한다면 이 코드를 쓰세요


# click_count = 0
# while True:
# #     if click_count == 3:
# #         break 
        
#     try:   
#         #더보기 클릭
#         driver.find_element_by_css_selector("#vue-profile > div.match-history-container.mt-3 > button").click()
#         time.sleep(1)
#         click_count += 1

#     except Exception as e:
#         pass
#         break

In [9]:
# 컨테이너의 정보들은, start가 3부터 시작해서, 1씩 더해지며 쌓인다.
# 각 컨테이너 마다 (+) 버튼 눌러주는 코드
start = 3
while True:
    try:
        driver.find_element_by_css_selector("#vue-profile > div.match-history-container.mt-3 > div:nth-child(" + str(start) + ") > div > div.match-history__expand-toggle").click()
        time.sleep(0.5)
        start += 1
    
    except Exception as e:
        pass
        break

<ipython-input-9-f5ee5972c586>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector("#vue-profile > div.match-history-container.mt-3 > div:nth-child(" + str(start) + ") > div > div.match-history__expand-toggle").click()


In [10]:
# 전체 펼친 html
summoner_html = driver.page_source
summoner_soup = BeautifulSoup(summoner_html, 'html.parser')

In [11]:
driver.quit()

In [12]:
summoner_games_html = summoner_soup.select('div.match-history__content')

In [13]:
# 불러오는 게임의 수
# games_len = len(summoner_games_html)
games_len = 13

In [14]:
# 각각의 정보들을 각 리스트에 담아준다.
for i in range(games_len):
    Level.append(int(list(summoner_games_html[i].select_one('div.champion').stripped_strings)[0]))
    CS.append(int(list(summoner_games_html[i].select_one('div.info').stripped_strings)[3]))
    Kill.append(int(list(summoner_games_html[i].select_one('div.kda').stripped_strings)[0]))
    Death.append(int(list(summoner_games_html[i].select_one('div.kda').stripped_strings)[2]))
    Assist.append(int(list(summoner_games_html[i].select_one('div.kda').stripped_strings)[4]))
    Wardplaced.append(int(list(summoner_games_html[i].select_one(' div.wards.mt-1').stripped_strings)[1]))
    visionWardplaced.append(int(list(summoner_games_html[i].select_one(' div.wards.mt-1').stripped_strings)[0]))
    WardKilled.append(int(list(summoner_games_html[i].select_one(' div.wards.mt-1').stripped_strings)[2]))
    
    #
    gl = list(summoner_games_html[i].select_one('span.mt-md-1'))[0].split(':')
    Gamelength.append(int(gl[0])*60 + int(gl[1]))

In [15]:
Gamelength 

[1104, 1618, 1476, 1903, 1532, 1939, 1824, 1422, 1456, 1766, 1858, 1800, 2340]

In [16]:
# 게임의 결과가 승리인지 패배인지 알려주는 html
# 각 게임의 '자세히보기' 버튼을 눌러야 나오는 정보들을 담는 코드

for i in range(games_len):
    summoner_games_html = summoner_soup.select('div.match-history__result')
    if list(summoner_games_html[i].stripped_strings)[0] == '패배':
        summoner_games_html = summoner_soup.select('div.match-history-detail__content')
        selector1 = '#vue-profile > div.match-history-container.mt-3 > div:nth-child(' + str(i+3) + ') > div.match-history-detail > div.match-history-detail__content > div > div > div.col-12.col-lg-6.border-e6-left-md > div > div.match-detail-overview__summoners > div.match-detail-overview__summoner.match-detail-overview__summoner--me > div > div.match-detail-overview__summoner__column.stats > div.wards > div:nth-child(2) > span'
        selector2 = '#vue-profile > div.match-history-container.mt-3 > div:nth-child(' + str(i+3) + ') > div.match-history-detail > div.match-history-detail__content > div > div > div.col-12.col-lg-6.border-e6-left-md > div > div.match-detail-overview__summoners > div.match-detail-overview__summoner.match-detail-overview__summoner--me > div > div.match-detail-overview__summoner__column.stats > div.graph > span'
    elif list(summoner_games_html[i].stripped_strings)[0] == '승리':
        summoner_games_html = summoner_soup.select('div.match-history-detail__content')
        selector1 = '#vue-profile > div.match-history-container.mt-3 > div:nth-child(' + str(i+3) + ') > div.match-history-detail > div.match-history-detail__content > div > div > div:nth-child(1) > div > div.match-detail-overview__summoners > div.match-detail-overview__summoner.match-detail-overview__summoner--me > div > div.match-detail-overview__summoner__column.stats > div.wards > div:nth-child(2) > span'
        selector2 = '#vue-profile > div.match-history-container.mt-3 > div:nth-child(' + str(i+3) + ') > div.match-history-detail > div.match-history-detail__content > div > div > div:nth-child(1) > div > div.match-detail-overview__summoners > div.match-detail-overview__summoner.match-detail-overview__summoner--me > div > div.match-detail-overview__summoner__column.stats > div.graph > span'
        
    gold = list(summoner_games_html[i].select_one(selector1).stripped_strings)[0]
    damage = list(summoner_games_html[i].select_one(selector2).stripped_strings)[0]
    if gold[-1] == '천':
        g = float(gold[:-1])
        g = int(g * 1000)
        Gold.append(g)
        
    elif gold[-1] == '만':
        g = float(gold[:-1])
        g = int(g * 10000)
        Gold.append(g)
        
    if ',' in damage:
        damage = damage.replace(',', '')
        damage = int(damage)
        Damage.append(damage)
    
    time.sleep(0.5)

In [17]:
# 내 평균 데이터들을 구하는 코드
# 각 데이터의 합을 (60/게임시간) 즉, 분으로 나누어서 나타낸다.

my_average_cs = round((sum(CS) * 60) / (sum(Gamelength)), 3)
my_average_kill = round((sum(Kill) * 60) / (sum(Gamelength)), 3)
my_average_death = round((sum(Death) * 60) / (sum(Gamelength)), 3)
my_average_Assist = round((sum(Assist) * 60) / (sum(Gamelength)), 3)
my_level = round((sum(Level) * 60) / (sum(Gamelength)), 3)
my_wardplaced = round((sum(Wardplaced) * 60) / (sum(Gamelength)), 3)
my_visionWardplaced = round((sum(visionWardplaced) * 60) / (sum(Gamelength)), 3)
my_wardkilled = round((sum(WardKilled) * 60) / (sum(Gamelength)), 3)
my_gold = round((sum(Gold) * 60) / (sum(Gamelength)), 3)
my_damage = round((sum(Damage) * 60) / (sum(Gamelength)), 3)

dataindex = ["Level", "CS", "Kill", "Death", "Assist", "wardPlaced", 
             "visionwardPlaced", "wardkilled", "Gold", "Damage"]
mydatacol = [my_level, my_average_cs, my_average_kill, my_average_death, 
                   my_average_Assist, my_wardplaced, my_visionWardplaced, my_wardkilled,
                  my_gold, my_damage]

In [18]:
# 데이터 프레임으로 그 정보들을 확인한다.
mydata = pd.DataFrame(mydatacol, index = dataindex).T
mydata.index = ['me']
mydata

,Level,CS,Kill,Death,Assist,wardPlaced,visionwardPlaced,wardkilled,Gold,Damage
me,0.504,5.393,0.256,0.185,0.351,0.319,0.185,0.15,423.36,702.927


In [19]:
# READ DATA
data = pd.read_csv("C:/Users/HEESU/[opgg데이터분석가]/11.22(small).csv", index_col = 0)

#원하는 데이터
want_data = ['level', 'minionsKilled', 'championsKilled', 'numDeaths', 'assists', 'wardPlaced',
             'visionWardsBoughtInGame', 'wardKilled', 'goldEarned', 'totalDamageDealtToChampions', 'gameLength']
data = data[want_data]

In [20]:
# 새로운 DataFrame 생성 후, 각 데이터들을 합쳐나간다. (분당 ~ 으로 계산)

df = pd.DataFrame()
for col in want_data:
    smalldf = pd.DataFrame(round((data[col]*60) / (data['gameLength']), 3), columns = [col])
    df = pd.concat([df, smalldf], axis = 1)
df = df[['level', 'minionsKilled', 'championsKilled', 'numDeaths', 'assists', 'wardPlaced',
             'visionWardsBoughtInGame', 'wardKilled', 'goldEarned', 'totalDamageDealtToChampions']]
df.columns = dataindex
data = pd.concat([df, mydata])

In [21]:
# 분당 데이터
data

,Level,CS,Kill,Death,Assist,wardPlaced,visionwardPlaced,wardkilled,Gold,Damage
0,0.482,0.763,0.120,0.281,0.883,0.201,0.000,0.161,311.478,273.191
1,0.522,4.054,0.040,0.281,0.361,0.241,0.000,0.040,297.351,498.903
2,0.522,4.856,0.201,0.321,0.401,0.361,0.040,0.000,391.746,810.943
3,0.482,1.846,0.161,0.361,0.843,0.482,0.080,0.080,348.682,815.839
4,0.522,4.094,0.120,0.120,0.241,0.161,0.080,0.000,302.729,402.823
...,...,...,...,...,...,...,...,...,...,...
19996,0.432,4.077,0.336,0.240,0.336,0.216,0.096,0.048,407.170,981.631
19997,0.432,5.803,0.408,0.120,0.192,0.360,0.072,0.336,472.398,1073.669
19998,0.432,1.367,0.336,0.144,0.288,0.528,0.144,0.096,466.451,789.952
19999,0.384,1.295,0.168,0.168,0.576,2.254,0.144,0.408,308.633,545.995


In [22]:
# 데이터를 rank함수를 통해 상위 %로 바꾼다 
data = data.rank(ascending = False, pct = True)
data

,Level,CS,Kill,Death,Assist,wardPlaced,visionwardPlaced,wardkilled,Gold,Damage
0,0.595145,0.878806,0.639568,0.222314,0.003300,0.739163,0.857232,0.214064,0.725039,0.831758
1,0.379181,0.535373,0.867232,0.222314,0.214464,0.647693,0.857232,0.696065,0.776911,0.501775
2,0.379181,0.452452,0.415529,0.130743,0.161817,0.301960,0.613144,0.897005,0.392180,0.151417
3,0.595145,0.641618,0.522999,0.072671,0.004775,0.181691,0.429229,0.489551,0.574621,0.147693
4,0.379181,0.532948,0.639568,0.757862,0.449328,0.827959,0.429229,0.897005,0.757862,0.649368
...,...,...,...,...,...,...,...,...,...,...
19996,0.854107,0.533698,0.168542,0.340283,0.255037,0.706815,0.366007,0.644518,0.330908,0.066997
19997,0.854107,0.303835,0.098695,0.757862,0.570446,0.303960,0.461977,0.028399,0.135543,0.043398
19998,0.854107,0.714739,0.168542,0.676166,0.343308,0.163242,0.212764,0.422029,0.148143,0.165092
19999,0.970026,0.732763,0.504300,0.597445,0.045823,0.000250,0.212764,0.011749,0.734813,0.430828


In [23]:
# 나의 데이터만을 추출한다.

mydata = data[data.index == 'me']
mydata = round(mydata, 3)
mydata *= 100

In [26]:
mydata

,Level,CS,Kill,Death,Assist,wardPlaced,visionwardPlaced,wardkilled,Gold,Damage
me,47.3,37.4,29.6,53.5,23.0,40.3,13.4,23.9,27.1,24.1
